In [14]:
from dotenv import load_dotenv
import os

load_dotenv()
groq_Api_Key = os.environ.get("GROQ_API_KEY")

from langchain_groq import ChatGroq

llm = ChatGroq(
    groq_api_key= groq_Api_Key,
               model="openai/gpt-oss-20b")

llm.invoke("SAY HEY IN SANSKRIT")


AIMessage(content='**है** (हे) – pronounced “he”\n\nIn Sanskrit, the interjection *हे* (he) is used just like “hey” in English to get someone’s attention or to greet informally. It’s the simplest, most direct equivalent.', additional_kwargs={'reasoning_content': 'The user says: "SAY HEY IN SANSKRIT". They want a translation of "Hey" into Sanskrit. Or maybe they want to say "Hey" in Sanskrit. "Hey" could be "हैलो" but in Sanskrit, a casual greeting might be "नमः" or "नमस्ते". "Hey" might be "हे" or "हे" is used as an interjection. In Sanskrit, "Hey" can be "हे" (he). But we need to respond appropriately. The user wants "SAY HEY IN SANSKRIT". So we should provide the Sanskrit word for "Hey". Provide the transliteration and maybe the Devanagari script. Also maybe mention usage. The user might just want a direct translation. So answer: "हे" (he). Or "नमः" or "नमस्ते". The simplest: "हे". So respond with that. Also mention it\'s used as an interjection. Provide transliteration.'}, response_